<a href="https://colab.research.google.com/github/DavidWilliamSmith/Python/blob/main/NBA_PER_FINAL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install nba_api
!pip install tqdm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 261.7/261.7 kB 5.0 MB/s eta 0:00:00


In [ ]:
from nba_api.stats.endpoints import leaguedashteamstats, playercareerstats
from nba_api.stats.static import players
import pandas as pd
import numpy as np
import time
import warnings
warnings.filterwarnings('ignore')
from nba_api.stats.endpoints import commonplayerinfo

def fetch_pace_data(season):
    team_stats = leaguedashteamstats.LeagueDashTeamStats(season=season, measure_type_detailed_defense='Advanced')
    team_data = team_stats.get_data_frames()[0]
    pace_data = team_data[['TEAM_NAME', 'PACE']]
    league_pace = pace_data['PACE'].mean()
    return pace_data, league_pace

def calculate_uPER(row):
    # Insert the detailed formula for uPER here
    uPER = (row['PTS'] + row['REB'] + row['AST'] - row['TOV']) / row['GP']
    return uPER

def calculate_aPER(uPER, team_pace, league_pace):
    aPER = uPER * (league_pace / team_pace)
    return aPER

def calculate_nPER(aPER, league_average_aPER):
    nPER = aPER * (15 / league_average_aPER)
    return nPER

# Function to get player position with a delay
def get_player_position(player_id):
    player_info = commonplayerinfo.CommonPlayerInfo(player_id=player_id)
    player_info_df = player_info.get_data_frames()[0]
    time.sleep(0.5)  # Half-second delay
    return player_info_df['POSITION'][0]

# Fetch active players and their positions
active_players_list = players.get_active_players()
active_players_df = pd.DataFrame(active_players_list)

#6 minutes
# Adding a position column to active_players_df
active_players_df['position'] = active_players_df['id'].apply(get_player_position)
active_players_df.head()

,id,full_name,first_name,last_name,is_active,position
0,1630173,Precious Achiuwa,Precious,Achiuwa,True,Forward
1,203500,Steven Adams,Steven,Adams,True,Center
2,1628389,Bam Adebayo,Bam,Adebayo,True,Center-Forward
3,1630534,Ochai Agbaji,Ochai,Agbaji,True,Guard
4,1630583,Santi Aldama,Santi,Aldama,True,Forward-Center


In [ ]:
from tqdm import tqdm

# Fetch player stats and team pace data
season = '2022-23'  # specify the season
team_pace_data, league_pace = fetch_pace_data(season)
all_players_stats_df = pd.DataFrame()

# about 7 minutes
# Loop with tqdm for progress indication
all_players_stats_list = []
for player in tqdm(active_players_list, desc="Processing Players"):
    player_stats = playercareerstats.PlayerCareerStats(player_id=player['id'])
    player_stats_df = player_stats.get_data_frames()[0]
    player_stats_df = player_stats_df[player_stats_df['SEASON_ID'] == '2022-23']  # Filter for 2022-23 season
    all_players_stats_list.append(player_stats_df)
    time.sleep(0.5)  # Delay to prevent rate limiting

all_players_stats_df = pd.concat(all_players_stats_list)
# Check if all_players_stats_df is empty
print("All Players Stats DataFrame:")
print(all_players_stats_df.head())

Processing Players: 100%|██████████| 531/531 [07:40<00:00,  1.15it/s]


All Players Stats DataFrame:
  PLAYER_ID SEASON_ID LEAGUE_ID     TEAM_ID TEAM_ABBREVIATION  PLAYER_AGE  GP  \
2   1630173   2022-23        00  1610612761               TOR        23.0  55   
9    203500   2022-23        00  1610612763               MEM        29.0  42   
5   1628389   2022-23        00  1610612748               MIA        25.0  75   
0   1630534   2022-23        00  1610612762               UTA        23.0  59   
1   1630583   2022-23        00  1610612763               MEM        22.0  77   

   GS     MIN  FGM  ... FT_PCT  OREB DREB  REB  AST STL BLK  TOV   PF   PTS  
2  12  1141.0  196  ...  0.702   100  228  328   50  31  30   59  102   508  
9  42  1133.0  157  ...  0.364   214  271  485   97  36  46   79   98   361  
5  75  2598.0  602  ...  0.806   184  504  688  240  88  61  187  208  1529  
0  22  1209.0  165  ...  0.812    43   78  121   67  16  15   41   99   467  
1  20  1682.0  247  ...  0.750    85  286  371   97  45  48   60  143   696  

[5 rows x 27 co

In [ ]:
team_name_mapping = {
    'ATL': 'Atlanta Hawks',
    'BOS': 'Boston Celtics',
    'BKN': 'Brooklyn Nets',
    'CHA': 'Charlotte Hornets',
    'CHI': 'Chicago Bulls',
    'CLE': 'Cleveland Cavaliers',
    'DAL': 'Dallas Mavericks',
    'DEN': 'Denver Nuggets',
    'DET': 'Detroit Pistons',
    'GSW': 'Golden State Warriors',
    'HOU': 'Houston Rockets',
    'IND': 'Indiana Pacers',
    'LAC': 'LA Clippers',
    'LAL': 'Los Angeles Lakers',
    'MEM': 'Memphis Grizzlies',
    'MIA': 'Miami Heat',
    'MIL': 'Milwaukee Bucks',
    'MIN': 'Minnesota Timberwolves',
    'NOP': 'New Orleans Pelicans',
    'NYK': 'New York Knicks',
    'OKC': 'Oklahoma City Thunder',
    'ORL': 'Orlando Magic',
    'PHI': 'Philadelphia 76ers',
    'PHX': 'Phoenix Suns',
    'POR': 'Portland Trail Blazers',
    'SAC': 'Sacramento Kings',
    'SAS': 'San Antonio Spurs',
    'TOR': 'Toronto Raptors',
    'UTA': 'Utah Jazz',
    'WAS': 'Washington Wizards'
}

# Apply the mapping
all_players_stats_df['TEAM_NAME'] = all_players_stats_df['TEAM_ABBREVIATION'].map(team_name_mapping)

# Merge with team pace data, player details, and position
# merged_df = pd.merge(all_players_stats_df, active_players_df[['id', 'full_name', 'position']], how='left', left_on='PLAYER_ID', right_on='id')
# merged_df = pd.merge(merged_df, team_pace_data[['TEAM_NAME', 'PACE']], how='left', left_on='TEAM_ABBREVIATION', right_on='TEAM_NAME')
# Now perform the merge
merged_df = pd.merge(all_players_stats_df, active_players_df[['id', 'full_name', 'position']], how='left', left_on='PLAYER_ID', right_on='id')
merged_df = pd.merge(merged_df, team_pace_data[['TEAM_NAME', 'PACE']], how='left', on='TEAM_NAME')

# Check merged DataFrame
print("Merged DataFrame:")
print(merged_df.head())

# Recalculate PER metrics
merged_df['uPER'] = merged_df.apply(calculate_uPER, axis=1)
merged_df['aPER'] = merged_df.apply(lambda row: calculate_aPER(row['uPER'], row['PACE'], league_pace) if pd.notnull(row['PACE']) else np.nan, axis=1)
league_average_aPER = merged_df['aPER'].mean(skipna=True)
merged_df['nPER'] = merged_df.apply(lambda row: calculate_nPER(row['aPER'], league_average_aPER) if pd.notnull(row['aPER']) else np.nan, axis=1)

# Final DataFrame with position
final_df = merged_df[['PLAYER_ID', 'full_name', 'position', 'PLAYER_AGE', 'uPER', 'aPER', 'nPER']]
print("Final DataFrame:")
final_df.head()

Merged DataFrame:
  PLAYER_ID SEASON_ID LEAGUE_ID     TEAM_ID TEAM_ABBREVIATION  PLAYER_AGE  GP  \
0   1630173   2022-23        00  1610612761               TOR        23.0  55   
1    203500   2022-23        00  1610612763               MEM        29.0  42   
2   1628389   2022-23        00  1610612748               MIA        25.0  75   
3   1630534   2022-23        00  1610612762               UTA        23.0  59   
4   1630583   2022-23        00  1610612763               MEM        22.0  77   

   GS     MIN  FGM  ... STL  BLK  TOV   PF   PTS          TEAM_NAME       id  \
0  12  1141.0  196  ...  31   30   59  102   508    Toronto Raptors  1630173   
1  42  1133.0  157  ...  36   46   79   98   361  Memphis Grizzlies   203500   
2  75  2598.0  602  ...  88   61  187  208  1529         Miami Heat  1628389   
3  22  1209.0  165  ...  16   15   41   99   467          Utah Jazz  1630534   
4  20  1682.0  247  ...  45   48   60  143   696  Memphis Grizzlies  1630583   

          full

,PLAYER_ID,full_name,position,PLAYER_AGE,uPER,aPER,nPER
0,1630173,Precious Achiuwa,Forward,23.0,15.036364,15.334735,15.319578
1,203500,Steven Adams,Center,29.0,20.571429,20.225194,20.205203
2,1628389,Bam Adebayo,Center-Forward,25.0,30.266667,31.214976,31.184123
3,1630534,Ochai Agbaji,Guard,23.0,10.406780,10.280240,10.270079
4,1630583,Santi Aldama,Forward-Center,22.0,14.337662,14.096347,14.082414


In [ ]:
print(all_players_stats_df['TEAM_ABBREVIATION'].unique())
print(team_pace_data['TEAM_NAME'].unique())


['TOR' 'MEM' 'MIA' 'UTA' 'MIN' 'TOT' 'MIL' 'CLE' 'NOP' 'ORL' 'NYK' 'POR'
 'WAS' 'PHX' 'DET' 'GSW' 'CHA' 'LAL' 'SAS' 'SAC' 'LAC' 'DAL' 'CHI' 'ATL'
 'IND' 'DEN' 'BKN' 'BOS' 'OKC' 'PHI' 'HOU']
['Atlanta Hawks' 'Boston Celtics' 'Brooklyn Nets' 'Charlotte Hornets'
 'Chicago Bulls' 'Cleveland Cavaliers' 'Dallas Mavericks' 'Denver Nuggets'
 'Detroit Pistons' 'Golden State Warriors' 'Houston Rockets'
 'Indiana Pacers' 'LA Clippers' 'Los Angeles Lakers' 'Memphis Grizzlies'
 'Miami Heat' 'Milwaukee Bucks' 'Minnesota Timberwolves'
 'New Orleans Pelicans' 'New York Knicks' 'Oklahoma City Thunder'
 'Orlando Magic' 'Philadelphia 76ers' 'Phoenix Suns'
 'Portland Trail Blazers' 'Sacramento Kings' 'San Antonio Spurs'
 'Toronto Raptors' 'Utah Jazz' 'Washington Wizards']


In [ ]:
final_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 546 entries, 0 to 545
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   PLAYER_ID   546 non-null    object 
 1   full_name   546 non-null    object 
 2   position    546 non-null    object 
 3   PLAYER_AGE  546 non-null    float64
 4   uPER        546 non-null    float64
 5   aPER        490 non-null    float64
 6   nPER        490 non-null    float64
dtypes: float64(4), object(3)
memory usage: 34.1+ KB


In [ ]:
print(final_df.columns)

Index(['PLAYER_ID', 'full_name', 'position', 'PLAYER_AGE', 'uPER', 'aPER',
       'nPER'],
      dtype='object')


In [ ]:
top_players_by_position = {}
positions = merged_df['position'].unique()

for position in positions:
    top_players = merged_df[merged_df['position'] == position].sort_values(by='nPER', ascending=False).head(5)
    top_players_by_position[position] = top_players

combined_df = pd.concat(top_players_by_position.values())

import plotly.express as px

fig = px.bar(combined_df, x='full_name', y='nPER', color='position', barmode='group',
             hover_data=['PLAYER_AGE', 'TEAM_ABBREVIATION'], title='Top 5 Players by nPER in Each Position')

fig.show()


In [ ]:
import plotly.express as px

fig = px.scatter(combined_df, x='PLAYER_AGE', y='nPER', color='position',
                 symbol='TEAM_ABBREVIATION',
                 title='Player Age vs Performance (nPER)')
fig.show()


In [ ]:
# Calculate team average nPER
team_avg_nPER = combined_df.groupby('TEAM_ABBREVIATION')['nPER'].mean().reset_index()

fig = px.bar(team_avg_nPER, x='TEAM_ABBREVIATION', y='nPER',
             title='Team Average nPER')
fig.show()


In [ ]:
# Load the CSV file containing player salaries
salary_df = pd.read_csv('https://raw.githubusercontent.com/fenago/datasets/main/nba_team_money202223.csv')

# Remove leading and trailing spaces from column names
salary_df.columns = salary_df.columns.str.strip()

# Rename columns for consistency
salary_df.rename(columns={'2022/23 ': 'Salary_2022_23', '2022/23(*)': 'Adjusted_Salary_2022_23'}, inplace=True)

# Assuming 'final_df', 'all_players_stats_df', and 'merged_df' are pre-loaded dataframes with player stats
# You would need to make sure 'final_df' and 'all_players_stats_df' are loaded with actual data before this step

# Merge salary data with the three dataframes using 'full_name' as the common column for merging
# enhanced_final_df = pd.merge(final_df, salary_df, how='left', on='full_name')
# enhanced_all_players_stats_df = pd.merge(all_players_stats_df, salary_df, how='left', on='full_name')
enhanced_merged_df = pd.merge(merged_df, salary_df, how='left', on='full_name')

# Renaming columns in the merged dataframes as required
# enhanced_final_df.rename(columns={'full_name': 'Full_Name'}, inplace=True)
# enhanced_all_players_stats_df.rename(columns={'full_name': 'Full_Name'}, inplace=True)
enhanced_merged_df.rename(columns={'full_name': 'Full_Name'}, inplace=True)

# Display the head of the merged dataframes to verify the merge and the renamed columns
# print(enhanced_final_df.head())
# print(enhanced_all_players_stats_df.head())
# Renaming the '2022/23' column to '2022_23_Salary'
enhanced_merged_df = enhanced_merged_df.rename(columns={'2022/23': '2022_23_Salary'})

# Verify the change
print(enhanced_merged_df.columns)

enhanced_merged_df.head()


Index(['PLAYER_ID', 'SEASON_ID', 'LEAGUE_ID', 'TEAM_ID', 'TEAM_ABBREVIATION',
       'PLAYER_AGE', 'GP', 'GS', 'MIN', 'FGM', 'FGA', 'FG_PCT', 'FG3M', 'FG3A',
       'FG3_PCT', 'FTM', 'FTA', 'FT_PCT', 'OREB', 'DREB', 'REB', 'AST', 'STL',
       'BLK', 'TOV', 'PF', 'PTS', 'TEAM_NAME', 'id', 'Full_Name', 'position',
       'PACE', 'uPER', 'aPER', 'nPER', '2022_23_Salary',
       'Adjusted_Salary_2022_23'],
      dtype='object')


,PLAYER_ID,SEASON_ID,LEAGUE_ID,TEAM_ID,TEAM_ABBREVIATION,PLAYER_AGE,GP,GS,MIN,FGM,...,TEAM_NAME,id,Full_Name,position,PACE,uPER,aPER,nPER,2022_23_Salary,Adjusted_Salary_2022_23
0,1630173,2022-23,00,1610612761,TOR,23.0,55,12,1141.0,196,...,Toronto Raptors,1630173,Precious Achiuwa,Forward,97.85,15.036364,15.334735,15.319578,"$2,840,160","$2,924,489"
1,203500,2022-23,00,1610612763,MEM,29.0,42,42,1133.0,157,...,Memphis Grizzlies,203500,Steven Adams,Center,101.50,20.571429,20.225194,20.205203,"$17,926,829","$18,459,108"
2,1628389,2022-23,00,1610612748,MIA,25.0,75,75,2598.0,602,...,Miami Heat,1628389,Bam Adebayo,Center-Forward,96.76,30.266667,31.214976,31.184123,"$30,351,780","$31,252,978"
3,1630534,2022-23,00,1610612762,UTA,23.0,59,22,1209.0,165,...,Utah Jazz,1630534,Ochai Agbaji,Guard,101.02,10.406780,10.280240,10.270079,"$3,918,360","$4,034,703"
4,1630583,2022-23,00,1610612763,MEM,22.0,77,20,1682.0,247,...,Memphis Grizzlies,1630583,Santi Aldama,Forward-Center,101.50,14.337662,14.096347,14.082414,NaN,NaN


In [ ]:
# Assuming enhanced_merged_df is your DataFrame
null_count = enhanced_merged_df['2022_23_Salary'].isnull().sum()
print("Number of null or NaN values in 2022_23_Salary:", null_count)


Number of null or NaN values in 2022_23_Salary: 57


In [ ]:
# Check for columns with missing values and count them
missing_value_counts = enhanced_merged_df.isnull().sum()
columns_with_missing_values = missing_value_counts[missing_value_counts > 0]

print("Columns with missing values and their counts:")
print(columns_with_missing_values)


Columns with missing values and their counts:
TEAM_NAME                  56
PACE                       56
aPER                       56
nPER                       56
2022_23_Salary             57
Adjusted_Salary_2022_23    57
dtype: int64


In [ ]:
import plotly.express as px

fig = px.scatter(enhanced_merged_df.dropna(subset=['nPER', '2022_23_Salary']),
                 x='2022_23_Salary', y='nPER', color='position',
                 hover_data=['Full_Name'], title='nPER vs. 2022_23_Salary by Position')
fig.show()


In [ ]:
avg_nPER_by_position = enhanced_merged_df.groupby('position')['nPER'].mean().reset_index()

fig = px.bar(avg_nPER_by_position, x='position', y='nPER', title='Average nPER by Position')
fig.show()


In [ ]:
fig = px.box(enhanced_merged_df.dropna(subset=['2022_23_Salary']),
             y='2022_23_Salary', color='position',
             title='2022_23 Salary Distribution by Position')
fig.update_traces(quartilemethod="inclusive") # or "exclusive", or "linear" by default
fig.show()


In [ ]:
# Remove the dollar sign and commas, then convert to float
enhanced_merged_df['2022_23_Salary'] = enhanced_merged_df['2022_23_Salary'].replace('[\$,]', '', regex=True).astype(float)

# Now you can run the visualizations as provided before

fig = px.scatter(enhanced_merged_df.dropna(subset=['nPER', '2022_23_Salary']),
                 x='nPER', y='2022_23_Salary', hover_data=['Full_Name'],
                 trendline='ols', title='nPER vs. 2022_23_Salary with Trend Line')
fig.show()


In [ ]:
# Convert salary to float after removing dollar signs and commas
enhanced_merged_df['2022_23_Salary'] = enhanced_merged_df['2022_23_Salary'].replace('[\$,]', '', regex=True).astype(float)

# Handle NaN values in 'nPER' and '2022_23_Salary' before calculating the ratio
enhanced_merged_df = enhanced_merged_df.dropna(subset=['nPER', '2022_23_Salary'])

# Now perform the division for the 'Salary_to_nPER_Ratio'
enhanced_merged_df['Salary_to_nPER_Ratio'] = enhanced_merged_df['2022_23_Salary'] / enhanced_merged_df['nPER']

# Plotting the histogram
fig = px.histogram(enhanced_merged_df, x='Salary_to_nPER_Ratio', nbins=30, title='Histogram of Salary to nPER Ratio')
fig.show()



In [ ]:
fig = px.scatter(enhanced_merged_df.dropna(subset=['nPER', '2022_23_Salary', 'MIN']),
                 x='nPER', y='2022_23_Salary', size='MIN', hover_data=['Full_Name'],
                 title='nPER vs. 2022_23_Salary Sized by Minutes Played')
fig.show()


In [ ]:
import plotly.express as px

# Remove non-numeric characters and convert to float for salary
enhanced_merged_df['2022_23_Salary'] = enhanced_merged_df['2022_23_Salary'].replace('[\$,]', '', regex=True).astype(float)

# Drop NaN values
enhanced_merged_df.dropna(subset=['nPER', '2022_23_Salary', 'position'], inplace=True)

# Calculate value index as nPER divided by salary (higher is better)
enhanced_merged_df['Value_Index'] = enhanced_merged_df['nPER'] / enhanced_merged_df['2022_23_Salary']

# Generate visuals for each position
for position in enhanced_merged_df['position'].unique():
    fig = px.scatter(
        enhanced_merged_df[enhanced_merged_df['position'] == position],
        x='nPER', y='2022_23_Salary',
        hover_data=['Full_Name'],
        title=f'nPER vs. 2022_23_Salary for {position} Position'
    )
    fig.show()

# Identify undervalued and overpriced players by position
for position in enhanced_merged_df['position'].unique():
    pos_df = enhanced_merged_df[enhanced_merged_df['position'] == position]

    # Top 10 undervalued players
    undervalued = pos_df.nlargest(10, 'Value_Index')

    # Top 10 overpriced players
    overpriced = pos_df.nsmallest(10, 'Value_Index')

    # Display the results
    print(f"Top 10 Undervalued Players in {position} Position:\n", undervalued[['Full_Name', 'nPER', '2022_23_Salary', 'Value_Index']])
    print(f"Top 10 Overpriced Players in {position} Position:\n", overpriced[['Full_Name', 'nPER', '2022_23_Salary', 'Value_Index']])



Top 10 Undervalued Players in Forward Position:
              Full_Name       nPER  2022_23_Salary  Value_Index
338      Justin Minaya   8.035710         35096.0     0.000229
94   Julian Champagnie  14.585571        508891.0     0.000029
441       Luka Samanic  14.661984        538317.0     0.000027
86          Jamal Cain   8.929374        508891.0     0.000018
31     Dominick Barlow   7.709051        508891.0     0.000015
382     Eugene Omoruyi  13.093092       1013119.0     0.000013
313          KJ Martin  18.564424       1782621.0     0.000010
120     Moussa Diabate   4.768071        508891.0     0.000009
343      Isaiah Mobley   4.573708        508891.0     0.000009
263        Braxton Key   1.663547        201802.0     0.000008
Top 10 Overpriced Players in Forward Position:
              Full_Name       nPER  2022_23_Salary   Value_Index
93   Julian Champagnie   0.000000        508891.0  0.000000e+00
43       Davis Bertans   6.153258      16000000.0  3.845786e-07
215     Richaun Ho

In [ ]:
import plotly.express as px

# Assuming enhanced_merged_df is already prepared
# Adding trendline to the scatter plots
for position in enhanced_merged_df['position'].unique():
    position_df = enhanced_merged_df[enhanced_merged_df['position'] == position]
    fig = px.scatter(
        position_df,
        x='nPER', y='2022_23_Salary',
        hover_data=['Full_Name'],
        color='Value_Index',  # Color by Value Index
        title=f'nPER vs. 2022_23_Salary for {position} Position',
        trendline='ols'  # Ordinary Least Squares regression line
    )
    fig.update_layout(
        xaxis_title="nPER",
        yaxis_title="2022/23 Salary ($)",
        coloraxis_colorbar=dict(
            title="Value Index"
        )
    )
    fig.show()


In [ ]:
import pandas as pd

# Assuming enhanced_merged_df is your DataFrame
null_count = enhanced_merged_df['2022_23_Salary'].isnull().sum()
print("Number of null or NaN values in 2022_23_Salary:", null_count)


Number of null or NaN values in 2022_23_Salary: 0


In [ ]:
enhanced_merged_df.columns

Index(['PLAYER_ID', 'SEASON_ID', 'LEAGUE_ID', 'TEAM_ID', 'TEAM_ABBREVIATION',
       'PLAYER_AGE', 'GP', 'GS', 'MIN', 'FGM', 'FGA', 'FG_PCT', 'FG3M', 'FG3A',
       'FG3_PCT', 'FTM', 'FTA', 'FT_PCT', 'OREB', 'DREB', 'REB', 'AST', 'STL',
       'BLK', 'TOV', 'PF', 'PTS', 'TEAM_NAME', 'id', 'Full_Name', 'position',
       'PACE', 'uPER', 'aPER', 'nPER', '2022_23_Salary',
       'Adjusted_Salary_2022_23', 'Salary_to_nPER_Ratio', 'Value_Index'],
      dtype='object')

In [ ]:
from nba_api.stats.endpoints import leaguedashplayerstats

def fetch_league_averages(season):
    player_stats = leaguedashplayerstats.LeagueDashPlayerStats(season=season, measure_type_detailed_defense='Base')
    player_data = player_stats.get_data_frames()[0]

    lgAST = player_data['AST'].sum() / player_data['GP'].sum()
    lgFG = player_data['FGM'].sum()
    lgPTS = player_data['PTS'].sum()
    lgFT = player_data['FTM'].sum()
    lgOREB = player_data['OREB'].sum()
    lgTOV = player_data['TOV'].sum()
    lgFTA = player_data['FTA'].sum()
    lgTRB = player_data['REB'].sum()

    return lgAST, lgFG, lgPTS, lgFT, lgOREB, lgTOV, lgFTA, lgTRB

#  usage
season = '2022-23'  # specify the season
lgAST, lgFG, lgPTS, lgFT, lgOREB, lgTOV, lgFTA, lgTRB = fetch_league_averages(season)

def calculate_uPER(row):
    factor = (2 / 3) - (0.5 * (lgAST / lgFG)) / (2 * (lgFG / lgFT))
    VOP = lgPTS / (lgFG - lgOREB + lgTOV + 0.44 * lgFTA)
    DRB_perc = (lgTRB - lgOREB) / lgTRB

    uPER = (1 / row['MIN']) * (
        row['FGM'] * VOP
        + 0.44 * row['FTA'] * VOP
        - row['FTA']
        + VOP * (row['REB'] - row['OREB'])
        + row['AST'] * VOP * factor
        + row['STL'] * VOP * DRB_perc
        - row['PF'] * ((row['FGA'] - row['FGM']) * (1 - DRB_perc) + row['FTA'] * 0.4 * (1 - DRB_perc))
        - row['TOV']
    )
    return uPER

def calculate_aPER(uPER, team_pace, league_pace):
    aPER = uPER * (league_pace / team_pace)
    return aPER

def calculate_nPER(aPER, league_average_aPER):
    nPER = aPER * (15 / league_average_aPER)
    return nPER




In [ ]:
# Calculate the updated uPER for each row
enhanced_merged_df['uPER'] = enhanced_merged_df.apply(calculate_uPER, axis=1)

# Calculate league average aPER
league_average_aPER = enhanced_merged_df['uPER'].mean()

# Calculate the updated aPER and nPER for each row
enhanced_merged_df['aPER'] = enhanced_merged_df.apply(lambda row: calculate_aPER(row['uPER'], row['PACE'], league_pace), axis=1)
enhanced_merged_df['nPER'] = enhanced_merged_df.apply(lambda row: calculate_nPER(row['aPER'], league_average_aPER), axis=1)

In [ ]:
import plotly.express as px

# Ensure that the '2022_23_Salary' column is in the correct numeric format
enhanced_merged_df['2022_23_Salary'] = enhanced_merged_df['2022_23_Salary'].replace('[\$,]', '', regex=True).astype(float)

# Create the box plot
fig = px.box(enhanced_merged_df, x='position', y=['2022_23_Salary', 'nPER'],
             title='Box Plot of Salary and nPER by Position',
             labels={'value':'Metric', 'variable':'Metric Type'})

fig.show()


In [ ]:
import plotly.express as px

# Assuming 'enhanced_merged_df' is already loaded and contains the columns '2022_23_Salary', 'nPER', and 'position'

# We need to ensure that salary data is in a numeric format without any non-numeric characters
enhanced_merged_df['2022_23_Salary'] = enhanced_merged_df['2022_23_Salary'].replace('[\$,]', '', regex=True).astype(float)

# Now let's create the scatter plot
fig = px.scatter(
    enhanced_merged_df,
    x='2022_23_Salary',
    y='nPER',
    color='position',  # Differentiates data points by position
    hover_data=['Full_Name'],  # Shows player name when you hover over points
    title='Scatter Plot of Salary and nPER by Position'
)

# Show the figure
fig.show()


In [ ]:
enhanced_merged_df.to_csv("enhanced_merged_df.csv")

In [ ]:
enhanced_merged_df.head()

,PLAYER_ID,SEASON_ID,LEAGUE_ID,TEAM_ID,TEAM_ABBREVIATION,PLAYER_AGE,GP,GS,MIN,FGM,...,Full_Name,position,PACE,uPER,aPER,nPER,2022_23_Salary,Adjusted_Salary_2022_23,Salary_to_nPER_Ratio,Value_Index
0,1630173,2022-23,00,1610612761,TOR,23.0,55,12,1141.0,196,...,Precious Achiuwa,Forward,97.85,-4.719290,-4.812936,16.424833,2840160.0,"$2,924,489",185394.145488,0.000005
1,203500,2022-23,00,1610612763,MEM,29.0,42,42,1133.0,157,...,Steven Adams,Center,101.50,-2.402841,-2.362399,8.062026,17926829.0,"$18,459,108",887238.267269,0.000001
2,1628389,2022-23,00,1610612748,MIA,25.0,75,75,2598.0,602,...,Bam Adebayo,Center-Forward,96.76,-11.961304,-12.336074,42.098618,30351780.0,"$31,252,978",973308.756315,0.000001
3,1630534,2022-23,00,1610612762,UTA,23.0,59,22,1209.0,165,...,Ochai Agbaji,Guard,101.02,-4.414841,-4.361160,14.883082,3918360.0,"$4,034,703",381531.621259,0.000003
5,1629638,2022-23,00,1610612762,UTA,24.0,36,3,528.0,83,...,Nickeil Alexander-Walker,Guard,101.02,-2.002438,-1.978090,6.750516,5009633.0,"$5,158,377",574677.051741,0.000002


In [ ]:
import pandas as pd
import plotly.express as px

# Load the enhanced merged dataframe
# enhanced_merged_df = pd.read_csv('/mnt/data/enhanced_merged_df.csv')

# We only want to consider players who have a salary and nPER listed
enhanced_merged_df.dropna(subset=['2022_23_Salary', 'nPER'], inplace=True)

# Scatter plot with nPER vs Salary and colored by position
scatter_fig = px.scatter(enhanced_merged_df,
                         x='nPER',
                         y='2022_23_Salary',
                         color='position',
                         hover_data=['Full_Name'],
                         title='nPER vs. Salary by Position')

# Box plot for nPER by position to get a sense of distribution
box_fig = px.box(enhanced_merged_df,
                 y='nPER',
                 x='position',
                 color='position',
                 title='Box Plot of nPER by Position')

# Calculate a simple metric for value: salary divided by nPER
# The lower the number, the more 'value' a player might be providing per their performance
enhanced_merged_df['Value_Metric'] = enhanced_merged_df['2022_23_Salary'] / enhanced_merged_df['nPER']

# Sorting the dataframe by the value metric to find the top 10 overvalued and undervalued players
overvalued_players = enhanced_merged_df.sort_values(by='Value_Metric', ascending=False).head(10)
undervalued_players = enhanced_merged_df.sort_values(by='Value_Metric', ascending=True).head(10)

scatter_fig.show()
box_fig.show()

overvalued_players[['Full_Name', 'position', 'nPER', '2022_23_Salary', 'Value_Metric']]
undervalued_players[['Full_Name', 'position', 'nPER', '2022_23_Salary', 'Value_Metric']]

overvalued_players.head()


,PLAYER_ID,SEASON_ID,LEAGUE_ID,TEAM_ID,TEAM_ABBREVIATION,PLAYER_AGE,GP,GS,MIN,FGM,...,position,PACE,uPER,aPER,nPER,2022_23_Salary,Adjusted_Salary_2022_23,Salary_to_nPER_Ratio,Value_Index,Value_Metric
26,1628964,2022-23,00,1610612747,LAL,25.0,9,1,88.0,11,...,Center,101.92,-0.113408,-0.111040,0.378939,10300000.0,"$10,605,825",1.353868e+06,7.386245e-07,2.718112e+07
478,202066,2022-23,00,1610612740,NOP,37.0,25,0,162.0,18,...,Guard-Forward,99.58,-0.080705,-0.080877,0.276004,5155500.0,"$5,308,575",1.693965e+06,5.903308e-07,1.867909e+07
435,1626156,2022-23,00,1610612747,LAL,27.0,17,17,526.0,107,...,Guard,101.92,-0.593919,-0.581516,1.984507,31377750.0,"$32,309,411",1.330091e+06,7.518280e-07,1.581136e+07
110,203109,2022-23,00,1610612749,MIL,32.0,18,3,340.0,45,...,Forward,101.45,-0.293235,-0.288442,0.984350,10183800.0,"$10,486,175",8.757699e+05,1.141852e-06,1.034571e+07
298,201567,2022-23,00,1610612748,MIA,34.0,21,17,419.0,54,...,Forward-Center,96.76,-0.888070,-0.915895,3.125622,30556968.0,"$31,464,258",2.089991e+06,4.784710e-07,9.776283e+06


In [ ]:
overvalued_players["Full_Name"].head(10)


26              Mo Bamba
478       Garrett Temple
435     D'Angelo Russell
110          Jae Crowder
298           Kevin Love
467       Jaden Springer
517    Russell Westbrook
287             Alex Len
51          Goga Bitadze
431         Derrick Rose
Name: Full_Name, dtype: object

In [ ]:
undervalued_players["Full_Name"].head(10)


514     Peyton Watson
312       Cody Martin
175       Danny Green
445      Olivier Sarr
388    Gary Payton II
132      Kevin Durant
249      Damian Jones
176       Danny Green
248      Damian Jones
480      Daniel Theis
Name: Full_Name, dtype: object

In [ ]:
import pandas as pd

# Load your dataframe
# enhanced_merged_df = pd.read_csv('/mnt/data/enhanced_merged_df.csv')

# Assuming '2022_23_Salary' is already converted to float and 'nPER' is calculated correctly
# Create a value score by dividing nPER by salary
enhanced_merged_df['Value_Score'] = enhanced_merged_df['nPER'] / enhanced_merged_df['2022_23_Salary']

# Now, sort the dataframe by 'Value_Score' and 'position' to find the top 10 undervalued and overvalued players
undervalued = enhanced_merged_df.sort_values(by=['position', 'Value_Score'], ascending=[True, False])
overvalued = enhanced_merged_df.sort_values(by=['position', 'Value_Score'], ascending=[True, True])

top_undervalued_by_position = undervalued.groupby('position').head(10)
top_overvalued_by_position = overvalued.groupby('position').head(10)

# Display results
print("Top Undervalued Players by Position:")
print(top_undervalued_by_position[['Full_Name', 'position', 'nPER', '2022_23_Salary', 'Value_Score']])

print("\nTop Overvalued Players by Position:")
print(top_overvalued_by_position[['Full_Name', 'position', 'nPER', '2022_23_Salary', 'Value_Score']])


Top Undervalued Players by Position:
              Full_Name       position       nPER  2022_23_Salary  Value_Score
158          Luka Garza         Center   7.707382        508891.0     0.000015
449      Alperen Sengun         Center  42.700625       3375360.0     0.000013
278        Jock Landale         Center  18.760273       1563518.0     0.000012
421       Nick Richards         Center  16.638558       1782621.0     0.000009
427    Orlando Robinson         Center   3.302110        386055.0     0.000009
..                  ...            ...        ...             ...          ...
283          Damion Lee  Guard-Forward  16.989174       2133278.0     0.000008
525      Jalen Williams  Guard-Forward  28.170383       4341480.0     0.000006
47           Saddiq Bey  Guard-Forward  18.582525       2959080.0     0.000006
317  Bennedict Mathurin  Guard-Forward  40.130873       6586800.0     0.000006
527    Kenrich Williams  Guard-Forward  11.138748       2000000.0     0.000006

[70 rows x 5 c

In [ ]:
import pandas as pd

# Load the merged dataframe containing all the enhanced player data including nPER and salary

# Filter the dataframe for players from the Miami Heat
miami_heat_players = enhanced_merged_df[enhanced_merged_df['TEAM_ABBREVIATION'] == 'MIA']

# Perform a descriptive analysis on the Miami Heat players
miami_heat_analysis = {
    'Total Players': miami_heat_players.shape[0],
    'Average Age': miami_heat_players['PLAYER_AGE'].mean(),
    'Average nPER': miami_heat_players['nPER'].mean(),
    'Total Salary': miami_heat_players['2022_23_Salary'].sum(),
    'Average Salary': miami_heat_players['2022_23_Salary'].mean(),
    'Highest Paid Player': miami_heat_players.loc[miami_heat_players['2022_23_Salary'].idxmax(), 'Full_Name'],
    'Top Performer by nPER': miami_heat_players.loc[miami_heat_players['nPER'].idxmax(), 'Full_Name'],
    'Salary of Top Performer by nPER': miami_heat_players.loc[miami_heat_players['nPER'].idxmax(), '2022_23_Salary'],
    'nPER of Highest Paid Player': miami_heat_players.loc[miami_heat_players['2022_23_Salary'].idxmax(), 'nPER'],
}

# Display the analysis
miami_heat_analysis


{'Total Players': 17,
 'Average Age': 27.352941176470587,
 'Average nPER': 13.158505155658789,
 'Total Salary': 176138273.0,
 'Average Salary': 10361074.88235294,
 'Highest Paid Player': 'Jimmy Butler',
 'Top Performer by nPER': 'Bam Adebayo',
 'Salary of Top Performer by nPER': 30351780.0,
 'nPER of Highest Paid Player': 16.698386466429064}

In [ ]:
# Assuming df['2022_23_Salary'] has already been cleaned and converted to float earlier in the script
# Calculate the Value Ratio for each player
miami_heat_df['Value_Ratio'] = miami_heat_df['nPER'] / miami_heat_df['2022_23_Salary']

# Now we can proceed to find the top 3 players by value for each position
top_players_by_position = {}
for position in miami_heat_df['position'].unique():
    top_players = miami_heat_df[miami_heat_df['position'] == position] \
        .sort_values(by='Value_Ratio', ascending=False) \
        .head(3)
    top_players_by_position[position] = top_players

top_players_by_position
import plotly.express as px

# Loop through each position and create a bar chart for the top 3 players by value
for position, players in top_players_by_position.items():
    fig = px.bar(players,
                 x='Full_Name',
                 y='Value_Ratio',
                 title=f'Top 3 Valued Players in Position: {position}')
    fig.show()


In [ ]:
# Create a value ratio for the entire league
df['Value_Ratio'] = df['nPER'] / df['2022_23_Salary']

# Get top 3 players by value ratio for each position in the league
top_players_league_by_position = {}
bottom_players_heat_by_position = {}

for position in df['position'].unique():
    top_players_league_by_position[position] = df[df['position'] == position].sort_values(by='Value_Ratio', ascending=False).head(3)
    bottom_players_heat_by_position[position] = miami_heat_df[miami_heat_df['position'] == position].sort_values(by='Value_Ratio', ascending=True).head(3)

# Code for visualization would be similar to the previous example but would include both top_players_league_by_position and bottom_players_heat_by_position


In [ ]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Assuming top_players_league_by_position and bottom_players_heat_by_position are dictionaries
# with position as keys and top 3 players dataframes as values

# Create subplots
fig = make_subplots(rows=2, cols=1, subplot_titles=("Top 3 Players by Position in the League", "Bottom 3 Players by Position on Miami Heat"))

# Add traces for top 3 in the league
for idx, position in enumerate(top_players_league_by_position.keys()):
    fig.add_trace(
        go.Bar(x=top_players_league_by_position[position]['Full_Name'],
               y=top_players_league_by_position[position]['Value_Ratio'],
               name=f"Top {position}"),
        row=1, col=1
    )

# Add traces for bottom 3 on the Heat
for idx, position in enumerate(bottom_players_heat_by_position.keys()):
    fig.add_trace(
        go.Bar(x=bottom_players_heat_by_position[position]['Full_Name'],
               y=bottom_players_heat_by_position[position]['Value_Ratio'],
               name=f"Bottom {position}"),
        row=2, col=1
    )

# Update layout
fig.update_layout(height=1200, width=800, title_text="Value Ratio by Position")

# Show figure
fig.show()


In [ ]:
df = enhanced_merged_df

# Add a column for value ratio
df['Value_Ratio'] = df['nPER'] / df['2022_23_Salary']

# League-Wide Analysis
top_3_league = df.groupby('position').apply(lambda x: x.nlargest(3, 'Value_Ratio')).reset_index(drop=True)
bottom_3_league = df.groupby('position').apply(lambda x: x.nsmallest(3, 'Value_Ratio')).reset_index(drop=True)

# Miami Heat Analysis
miami_heat_df = df[df['TEAM_ABBREVIATION'] == 'MIA']
top_3_heat = miami_heat_df.groupby('position').apply(lambda x: x.nlargest(3, 'Value_Ratio')).reset_index(drop=True)
bottom_3_heat = miami_heat_df.groupby('position').apply(lambda x: x.nsmallest(3, 'Value_Ratio')).reset_index(drop=True)


In [ ]:
# Assuming 'enhanced_merged_df' is your DataFrame with nPER and salary data
# Make sure the '2022_23_Salary' column is in float format
enhanced_merged_df['2022_23_Salary'] = enhanced_merged_df['2022_23_Salary'].replace('[\$,]', '', regex=True).astype(float)

# Calculate the Value Ratio
enhanced_merged_df['Value_Ratio'] = enhanced_merged_df['nPER'] / enhanced_merged_df['2022_23_Salary']

# Sort by Value Ratio in ascending order to find the most overvalued players
overvalued_players = enhanced_merged_df.sort_values(by='Value_Ratio').head(15)

# Print the top 5 overvalued players
print(overvalued_players[['Full_Name', 'nPER', '2022_23_Salary', 'Value_Ratio']])


             Full_Name      nPER  2022_23_Salary  Value_Ratio
163      Jacob Gilyard -1.744888          5849.0    -0.000298
493      Stanley Umude -2.385368         58493.0    -0.000041
320        Skylar Mays -1.262593        116574.0    -0.000011
412    Lester Quinones -1.310632        129405.0    -0.000010
338      Justin Minaya -0.328495         35096.0    -0.000009
523  Lindell Wigginton -0.786289         99438.0    -0.000008
263        Braxton Key -1.504061        201802.0    -0.000007
403       Micah Potter -3.032437        508891.0    -0.000006
221           Jay Huff -0.550736        116986.0    -0.000005
410      Neemias Queta -2.173512        508891.0    -0.000004
472        Cole Swider -1.618577        508891.0    -0.000003
117         JD Davison -1.618352        508891.0    -0.000003
343      Isaiah Mobley -1.402983        508891.0    -0.000003
71       Kendall Brown -1.393159        508891.0    -0.000003
460          Dru Smith -1.000353        374357.0    -0.000003


In [ ]:
# Filter players who have played more than 41 games
players_over_41_games = enhanced_merged_df[enhanced_merged_df['GP'] > 41]

# Sort by Value Ratio in ascending order for overvalued players
overvalued_players = players_over_41_games.sort_values(by='Value_Ratio').head(15)

# Sort by Value Ratio in descending order for undervalued players
undervalued_players = players_over_41_games.sort_values(by='Value_Ratio', ascending=False).head(15)

# Print the top 15 overvalued and undervalued players
print("Top 15 Overvalued Players:")
print(overvalued_players[['Full_Name', 'nPER', '2022_23_Salary', 'Value_Ratio']])
print("\nTop 15 Undervalued Players:")
print(undervalued_players[['Full_Name', 'nPER', '2022_23_Salary', 'Value_Ratio']])


Top 15 Overvalued Players:
             Full_Name       nPER  2022_23_Salary   Value_Ratio
253         Tyus Jones   3.026562      15000000.0  2.017708e-07
215     Richaun Holmes   2.697906      11215260.0  2.405567e-07
455        Ben Simmons   9.648759      35448672.0  2.721896e-07
167        Eric Gordon   6.570806      19568360.0  3.357873e-07
206     Gordon Hayward  10.663908      30075000.0  3.545772e-07
331  Jordan McLaughlin   0.835588       2160000.0  3.868461e-07
216         Al Horford  10.978851      26500000.0  4.142963e-07
288      Kawhi Leonard  17.814225      42492492.0  4.192323e-07
43       Davis Bertans   6.890775      16000000.0  4.306735e-07
84        Jimmy Butler  16.698386      37653300.0  4.434774e-07
1         Steven Adams   8.062026      17926829.0  4.497184e-07
238       LeBron James  22.562625      44474988.0  5.073104e-07
457       Jericho Sims   0.870059       1639842.0  5.305747e-07
426  Mitchell Robinson   9.460942      17045454.0  5.550420e-07
38        Bra